In [1]:
# vfinal (incompressible: moodify solution of the base flow) Base flow with continuous elements
# Artificial viscosity

# 1/5

# SIMPLE two-fluid model: Holmas, Bonzanini (main)

# The interfacial and wall shear stresses in (A.2) are modelled using standard single-phase pipe friction relations (Haaland, 1983).
# https://fenicsproject.org/pub/tutorial/html/._ftut1017.html

## with Shear
"""
Two-fluid model
"""
# Libraries for stability analysis
from dolfin import *
from IPython.display import clear_output
from scipy.optimize import brenth
from scipy.optimize import fsolve
from scipy.linalg import eig
from scipy.sparse.linalg import eigs
from scipy.interpolate import interp1d
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from matplotlib import rc
from math import pi
from scipy.sparse import csr_matrix
from numpy import linalg as LA
from scipy import sparse

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
import time
import math

%matplotlib inline

# Phase properties
c2               = 293.43                 # Sanderse 2017
u4eq             = 1e5                    # Pa, outlet pressure (equilibrium state)
rho1             = 1000                   # kg m^-3, liquid density (Montini)
rho2             = u4eq/pow (c2, 2)       # kg m^-3, gas density (Sanderse)
mu1              = 8.9e-4                 # Pa s, liquid viscosity
mu2              = 1.8e-5                 # Pa s, gas viscosity
sigma            = 0.072                  # Montini 2011
rho              = (rho1 - rho2)/rho1     # Bonzanini 2017
# Gravity
g                = 9.8 # m/s^2

# Configuration
d                = 0.022                  # 0.022              # m, diameter
L                = 1                      # m, pipe length
beta             = 0                    # (Taitel 1976)
Area             = pi*pow (abs(d), 2.)/4.      # mˆ2, total area
S                = pi*d
roughness        = 1e-8                   # m, pipe roughness

# superficial velocities (check well-posedness [as a function])
j1         = 0.5                #0.80 # liquid superficial velocity
j2         = 6.908               # 1.53 # gas superficial velocity
us         = j1 + j2            # velocity of the mixture
Re1s       = rho1*j1*d/mu1

# artifical diffusion coefficients (depend on superficial velocities and stability analysis)
E11        = 7e-4               #0.001 
E22        = 7e-3               #0.01

# FUNCTIONS
# stratification angle
def gamma (u1):
    return 2*(pi*(1. - u1) + pow (abs(3.*pi/2.), (1./3.))*(2.*u1 - 1. + pow (abs (1. - u1), (1./3.))- pow (abs(u1), (1/3))) - (1./200.)*u1*(1. - u1)*(2.*u1 - 1.)*(1. + 4.*(pow (abs (u1), 2.) + pow (abs(1.- u1), 2.))))
# liquid holdup
def alpha1 (u1):
    return 1 - u1

# liquid phase velocity
def u3 (u1):
    return j1/(1. - u1)
# liquid phase velocity
def u3b (j1):
    return j1/(1. - u1)
# gas phase velocity
def u2 (u1):
    return j2/u1

# liquid sectional area
def A1 (gamma):
    return (d**2)/4*(gamma/2 - np.sin (gamma/2)*np.cos (gamma/2))
# gas sectional area
def A2 (gamma):
    return (d**2)/4*(pi - gamma/2 + np.sin (gamma/2)*np.cos (gamma/2))
# liquid wetted perimeter
def S1 (gamma):
    return d*gamma/2.
# gas wetted perimeter
def S2 (gamma):
    return d/2.*(2.*pi - gamma)
# interface wetted perimeter
def Si (gamma):
    return d*np.sin (gamma/2)

# liquid hydraulic diameter 
def dh1 (A1, S1):
    return 4.*A1/S1
# gas hydraulic diameter 
def dh2 (A2, S2, Si):
    return 4.*A2/(S2 + Si)
# critic diameter
def Dc (gamma):
    return g*np.cos (beta)*pi*d/(4*np.sin (gamma/2))

# liquid Reynolds number
def Re1 (u3, dh1):
    return rho1*u3*dh1/mu1
# gas Reynolds number
def Re2 (u2, dh2):
    return u4eq/pow ((c2), 2)*u2*dh2/mu2

# liquid shear stress
def tauw1 (f1, u3):
    return f1*rho1*u3*abs (u3)/2.
# gas shear stress
def tauw2 (f2, u2):
    return f2*u4eq/pow (c2, 2)*u2*abs (u2)/2.
# interface shear stress
def tau21 (fi, u3, u2):
    return fi*u4eq/pow (c2, 2)*(u2 - u3)*abs (u2 - u3)/2.
# interface shear stress
def tau12 (fi, u3, u2):
    return fi*u4eq/pow (c2, 2)*(u3 - u2)*abs (u3 - u2)/2.

# liquid friction factor 1
def f1 (Re1, dh1):
    Af1 = pow (2.457*np.log(pow(pow(abs(7/Re1), 0.9) + 0.27*roughness/dh1, - 1)), 16)
    Bf1 = pow (abs (37530/Re1), 16)
    return 2*pow (pow (abs (8/Re1), 12)+pow (abs (Af1 + Bf1), - 1.5), 1/12)

# gas friction factor 2
def f2 (Re2, dh2):
    Af2 = pow (2.457*np.log(pow(pow(abs(7/Re2), 0.9) + 0.27*roughness/dh2, - 1)), 16)
    Bf2 = pow (abs (37530/Re2), 16)
    return 2*pow (pow (abs (8/Re2), 12)+pow (abs (Af2 + Bf2), - 1.5), 1/12)

# interfacial friction
def fi (f2):
    return max (f2, 0.014)

# steady state momentum equation
def equilibrium1 (u1): 
    func = - (rho1 - u4eq/pow (c2, 2))*g*np.sin (beta) - tauw1 (f1 (Re1 (u3 (u1), dh1 (A1 (gamma (u1)), S1 (gamma (u1)))), dh1 (A1 (gamma (u1)), S1 (gamma (u1)))), u3 (u1))*S1 (gamma (u1))/A1 (gamma(u1)) + tauw2 (f2 (Re2 (u2 (u1), dh2 (A2 (gamma (u1)), S2 (gamma (u1)), Si (gamma (u1)))), dh2 (A2 (gamma (u1)), S2 (gamma (u1)), Si (gamma (u1)))), u2 (u1))*S2 (gamma (u1))/A2(gamma (u1)) + tau21 (fi (f2 (Re2 (u2 (u1), dh2 (A2 (gamma (u1)), S2 (gamma (u1)), Si(gamma (u1)))), dh2 (A2 (gamma (u1)), S2 (gamma (u1)), Si (gamma (u1))))), u3 (u1), u2 (u1))*Si(gamma (u1))*((1/A1 (gamma (u1))) + (1/A2 (gamma (u1))))
    return func
print ("Local equilibrium theory")

lima    = 1e-15
limb    = 1 - lima
u1eq    = brenth (equilibrium1, lima, limb)
print("alpha2eq = ", u1eq)

# finding the solution with fsolve
# x0         = 0.01  #initial guess
# alpha2eq   = fsolve (equilibrium, x0) #Python function

# print ("Re_1     = ", Re1 (u1 (alpha1 (alpha2eq)), Dh1 (A1 (gamma (alpha2eq)), S1 (gamma (alpha2eq)))))
# print ("Re_2     = ", Re2 (u2 (alpha1 (alpha2eq)), Dh2 (A2 (gamma (alpha2eq)), S2 (gamma (alpha2eq)), Si (gamma (alpha2eq)))))
# print (" ")

# parameters used in boundary conditions
u1_0     = u1eq
u2_0     = u2 (u1eq)
u3_0     = u3 (u1eq)
u4_0     = u4eq

# messages for boundary conditions just at the entrance, but not the actual base flow)
print ("Initial conditions phi (t, x) for both incompressible phases")
print ("Gas void fraction    : u1_0 (0, x) = ", u1_0, "[-]")
print ("Gas velocity         : u2_0 (0, x) = ", u2_0, "[m/s]")
print ("Liquid velocity      : u3_0 (0, x) = ", u3_0, "[m/s]")
print ("Interfacial pressure : u4_0 (0, x) = ", u4_0, "Pa")

print (" ")

print ("Boundary conditions phi (t, x) for both incompressible phases")
print ("Gas void fraction    : u1_0 (t, 0) = ", u1_0, "[-]")
print ("Gas velocity         : u2_0 (t, 0) = ", u2_0, "[m/s]")
print ("Liquid velocity      : u3_0 (t, 0) = ", u3_0, "[m/s]")
print ("Interfacial pressure : u4_0 (t, L) = ", u4_0, "Pa")

ModuleNotFoundError: No module named 'dolfin'

In [2]:
# 2/5

# FEniCS configuration
# #set_log_level(1) # Progress: to get as much information as possible we will set the log level to 1, but it can be 16
# set_log_level (LogLevel.TRACE)

# # Test for PETSc or Tpetra
# if not has_linear_algebra_backend ("PETSc") and not has_linear_algebra_backend ("Tpetra"):
#     info ("DOLFIN has not been configured with Trilinos or PETSc. Exiting.")
#     exit ()

# if not has_krylov_solver_preconditioner ("amg"):
#     info ("Sorry, this demo is only available when DOLFIN is compiled with AMG "
#          "preconditioner, Hypre or ML.")
#     exit ()

# if has_krylov_solver_method ("minres"):
#     krylov_method = "minres"
# elif has_krylov_solver_method ("tfqmr"):
#     krylov_method = "tfqmr"
# else:
#     info ("Default linear algebra backend was not compiled with MINRES or TFQMR "
#          "Krylov subspace method. Terminating.")
#     exit ()

# # Choose linear algebra backend    
# parameters["linear_algebra_backend"] = "PETSc"

# # Form compiler options
# parameters ["form_compiler"]["optimize"]          = True
# parameters ["form_compiler"]["cpp_optimize"]      = True
#parameters["form_compiler"]["representation"] = "quadrature"
parameters ["form_compiler"]["quadrature_degree"] = 2

# parameters ["form_compiler"]["cpp_optimize"]      = True
# ffc_options = {"optimize"               : True, \
#                "eliminate_zeros"        : True, \
#                "precompute_basis_const" : True, \
#                "precompute_ip_const"    : True}

# Adaptation two-fluid model (Bonzanini & Holmas)

# Define constants
var1bc1   = 1 - u1_0                    # alpha1
var2bc1   = u3_0                        # u1

# Define space discretization properties
xmin      = 0
xmax      = L
nx        = 100                           # no. of elements 100 to 1600
p         = 1                              # degree of FEM
mesh      = IntervalMesh (nx, xmin, xmax)
# h         = CellDiameter (mesh)
# hmin      = mesh.hmin ()
# deltax    = d/2.                           # D/2 Bonzanini #0.256*D holmas

# Save mesh
File ("tfm_1/mesh.xml") << mesh

# Define time discretization properties
# T         = 10.0                        # final time Holmas
# CFL       = 0.95                        # 0.7 Holmas, 0.978 Van Zwieten
# dt        = CFL*deltax/(var2bc1)        # Van Zwieten & Smith /lambda_max Holmas (maximum modulus of the eigenvalues of the Jacobian of (4.1) at time step n)
# num_steps = int( round (T/dt))
                
# print('T         =', T)
# print('dt        =', dt)
# print('num_steps =', num_steps)

# Define funcion spaces
# cell       = interval
# etypevar1  = 'Lagrange'
# etypevar2  = 'Lagrange'
V1         = FiniteElement ('Lagrange', mesh.ufl_cell(), degree = p)
V2         = FiniteElement ('Lagrange', mesh.ufl_cell(), degree = p)
element    = MixedElement ([V1, V2])
V          = FunctionSpace (mesh, element)

# Define test and trial functions
v1, v2     = TestFunctions (V)

var        = Function (V, name = "Variables at current step")
var1, var2 = split (var)

dvar       = TrialFunction (V)

# Define initial condition
class InitialConditions (UserExpression):
    def __init__ (self, **kwargs):
        super (InitialConditions, self).__init__(**kwargs)
    def eval (self, values, x):
        values[0] = var1bc1
        values[1] = var2bc1
    def value_shape (self):
        return (2,)
    
var_ic         = InitialConditions (degree = p)
var_n          = interpolate (var_ic, V) 
var1_n, var2_n = split (var_n)

# Plot initial conditions
plt.figure (1, figsize = (8, 4))
plt.grid (True, which = "both")
plt.xlim (xmin, xmax)
plot (var1_n, wireframe = True, title = "Initial liquid holdup")

plt.figure (2, figsize = (8, 4))
plt.grid (True, which = "both")
plt.xlim (xmin, xmax)
plot (var2_n, wireframe = True, title = "Initial liquid velocity")

plt.show

# Define boundary condition
var1_bc = Expression ("var1bc1", degree = p, var1bc1 = var1bc1)
var2_bc = Expression ("var2bc1", degree = p, var2bc1 = var2bc1)

# Sub domain for Dirichlet boundary condition
def right (x, on_boundary): 
    return x[0] > (1.0 - DOLFIN_EPS)
def left (x, on_boundary): 
    return x[0] < DOLFIN_EPS

bc1     = DirichletBC(V.sub (0), var1_bc, left)
bc3     = DirichletBC(V.sub (1), var2_bc, left)
bcs     = [bc1, bc3]

# Define expressions used in weak form
# k       = Expression ("dt", degree = p, dt = dt)
E11     = Expression ("E11", degree = p, E11 = E11)
E22     = Expression ("E22", degree = p, E22 = E22)


NameError: name 'parameters' is not defined

In [3]:
# 3/5

Area = Constant(Area)
rho1 = Constant(rho1)
rho2 = Constant(rho1)
beta = Constant(beta)
rho  = Constant(rho)
g    = Constant(g)

# Define functions used in weak form
def A11 (var1, var2):
    term1 = var2
    return term1

def A12 (var1, var2):
    term2 = var1
    return term2

def A21 (var1, var2):
    alpha2 = 1 - var1
    gamma  = 2*(pi*(1. - alpha2) + pow (abs(3.*pi/2.), (1./3.))*(2.*alpha2 - 1. + pow (abs (1. - alpha2), (1./3.))- pow (abs(alpha2), (1/3))) - (1./200.)*alpha2*(1. - alpha2)*(2.*alpha2 - 1.)*(1. + 4.*(pow (abs (alpha2), 2.) + pow (abs(1.- alpha2), 2.))))
    Si     = d*sin (gamma/2)
    term3  = 1/(1 - rho*var1)*(Area*cos (beta)*(1 - var1)*g*rho/Si + (rho - 1)* pow (abs(var2), 2) - ((rho2/rho1)*(us - var1*var2)*(us - (2 - var1)*var2))/pow (abs(1 - var1), 2))
    return term3
    
def A22 (var1, var2):
    term4  = 1/(1 - rho*var1)*((var2*(1 - 2*var1+rho*var1)) - ((2*rho2*var1*(us - var1*var2)/rho1)/(1 - var1)))
    return term4

def D2 (var1, var2):
    alpha2 = 1 - var1
    gamma  = 2*(pi*(1. - alpha2) + pow (abs(3.*pi/2.), (1./3.))*(2.*alpha2 - 1. + pow (abs (1. - alpha2), (1./3.))- pow (abs(alpha2), (1/3))) - (1./200.)*alpha2*(1. - alpha2)*(2.*alpha2 - 1.)*(1. + 4.*(pow (abs (alpha2), 2.) + pow (abs(1.- alpha2), 2.))))
    
    S1    = d*gamma/2
    S2    = d/2*(2*pi - gamma)
    Si    = d*sin (gamma/2)
    
    dh1   = 4*var1*Area/S1
    dh2   = 4*(alpha2)*Area/(S2 + Si)
    
    vel2  = (us - var1*var2)/(1 - var1)
    
    Re1   = rho1*var2*dh1/mu1
    Re2   = rho2*vel2*dh2/mu2
    
    Af1 = pow (2.457*ln(pow(pow(abs(7/Re1), 0.9) + 0.27*roughness/dh1, - 1)), 16)
    Bf1 = pow (abs (37530/Re1), 16)
    f1  = 2*pow (pow (abs (8/Re1), 12)+pow (abs (Af1 + Bf1), - 1.5), 1/12)
    
    Af2 = pow (2.457*ln(pow(pow(abs(7/Re2), 0.9) + 0.27*roughness/dh2, - 1)), 16)
    Bf2 = pow (abs (37530/Re2), 16)
    f2  = 2*pow (pow (abs (8/Re2), 12)+pow (abs (Af2 + Bf2), - 1.5), 1/12)
    
    fi  = f2 # max_value(f2, 0.014)
        
    tau1  = f1*rho1*var2*abs (var2)/2.
    tau2  = f2*rho2*vel2*abs (vel2)/2.
    taui  = fi*rho2*(vel2 - var2)*abs (vel2 - var2)/2.
    
    term5 = 1/(1 - rho*var1)*((var1*(tau2*S2 - tau1*S1) + taui*Si)/(rho1*var1*Area) + (1 - var1)*g*rho*sin(beta))
    return term5

# Define weak form (semi distretised with artificial difussion)
F = - (A11 (var1, var2)*v1*Dx(var1,0))*dx - A12 (var1, var2)*(v1*Dx(var2,0))*dx \
+ (inner(E11*grad(var1), grad(v1)))*dx - A21 (var1, var2)*v2*Dx(var1,0)*dx \
- (A22 (var1, var2)*v2*Dx(var2,0))*dx + inner(E22*grad(var2), grad(v2))*dx + D2 (var1, var2)*v2*dx

# Auxiliary
L = Constant(0)*v1*dx + Constant(0)*v2*dx

# Define Jacobian
dF = derivative (F, var, dvar)

# Assemble stiffness form    
A = PETScMatrix()
b = PETScVector()

assemble_system(dF, L, bcs, A_tensor = A, b_tensor = b)

A_mat = as_backend_type(A).mat()

# Transform to numpy array
A_sparray = csr_matrix(A_mat.getValuesCSR()[::-1], shape = A_mat.size)

# Print array
print(A_sparray)

# Plot array
plt.figure (3, figsize = (12, 8))
plt.spy(A_sparray)
plt.grid (True, which = "both")
# plt.rcParams ['figure.figsize'] = [12, 8]

plt.show


NameError: name 'Constant' is not defined

In [4]:
# 4/5

A_sparray = A_sparray.todense()
A_sparray = np.nan_to_num (A_sparray)
A_sparray = sparse.csr_matrix(A_sparray)

# Print array
print(A_sparray)


NameError: name 'A_sparray' is not defined

In [5]:
# 5/5

Acomplex_sparse = A_sparray.dot (1j)

Aeval           = A_sparray.toarray ()
Aeval           = np.nan_to_num (Aeval)

Acomplex        = Aeval.dot (1j)

condnumber = LA.cond(Aeval)
print("Condition number :", condnumber)

#The array v of eigenvectors may not be of maximum rank, 
#that is, some of the columns may be linearly dependent, 
#although round-off error may obscure that fact. 
#If the eigenvalues are all different, then theoretically the eigenvectors are linearly independent.
# TUTORIALS
# https://fenicsproject.org/qa/13796/formulating-an-eigenvalue-problem/
# https://fenicsproject.org/qa/8680/solving-generalized-eigenvalues-finite-element-stiff-matrix/

vals, vecs = eig ( - Acomplex, overwrite_a = True)

# vals, vecs = eigs ( - Acomplex_sparse) #tol = 1e-10

# Eigenvalues
imagvals = vals.imag
realvals = vals.real

print("realvals = ", vals.real)
print("imagvals = ", vals.imag)

# Characteristics for plots
liststyles       = ["--", "-", "-.", "."]
listcolor        = ["k", "g", "b", "r", "none"]
listmarkers      = ["s", "o", "^", ">", "<", "p"]

# Plot eigenspectra
fig, ax = plt.subplots ()
area = 50

ax.scatter (realvals, imagvals, s = area, marker = listmarkers [0], color = listcolor [4], edgecolors = listcolor [0], linewidths = 1.5, alpha = 0.5)
ax.set_xscale ('symlog', linthreshx = 1e-5)

plt.rcParams ['figure.figsize'] = [12, 8]
# leg1 = ax.legend (loc = 'upper right', frameon = True, fontsize = 14);
plt.grid (True, which = "both")
plt.xlabel ('Re $[\omega]$ [1/s]', fontsize = 18)
plt.ylabel ('Im $[\omega]$ [1/s]', fontsize = 16)
        
matplotlib.rc ('xtick', labelsize = 18)     
matplotlib.rc ('ytick', labelsize = 18)

# plt.ylim (( - 0.002 , 0.002))
# plt.xlim (( 1e-3, 1e1))
plt.show ()    

# Eigenvectors
localmaxreal = np.where(vals.real == vals.real.max())
localmaximag = np.where(vals.imag == vals.imag.max())

print("localmaxreal =", localmaxreal)
print("localmaximag =", localmaximag)

eigenvector_real = vecs.real[localmaxreal[0]]
eigenvector_imag = vecs.imag[localmaximag[0]]

print("eigenvector_real =", eigenvector_real[0])
print("eigenvector_imag =", eigenvector_imag[0])


# Plot real eigenvectors
ureal  = Function(V)
ureal.vector()[:] = eigenvector_real[0] #len(vecs)-1 
u1real, u2real = ureal.split()

# Plot eigenfunction
plt.figure (3, figsize = (12, 8))
plot(u1real, wireframe = True, title = 'Eigenfunction')
plt.grid (True, which = "both")
plt.xlabel("L")
plt.ylabel("rx (Eigenvector of the largest real eigenvalue)")

# Plot eigenfunction
plt.figure (3, figsize = (12, 8))
plot(u2real, wireframe = True, title = 'Eigenfunction')
plt.grid (True, which = "both")
plt.xlabel("L")
plt.ylabel("rx (Eigenvector of the largest real eigenvalue)")

# Plot imaginary eigenvectors
uimag = Function(V)
uimag.vector()[:] = eigenvector_imag[0] #eigenfunction)
u1imag, u2imag = uimag.split()

# Plot eigenfunction
plt.figure (4, figsize = (12, 8))
plot(u1imag, wireframe = True, title = 'Eigenfunction')
plt.grid (True, which = "both")
plt.xlabel("L")
plt.ylabel("cx (Eigenvector of the largest imaginary eigenvalue)")

plt.figure (4, figsize = (12, 8))
plot(u2imag, wireframe = True, title = 'Eigenfunction')
plt.grid (True, which = "both")
plt.xlabel("L")
plt.ylabel("cx (Eigenvector of the largest imaginary eigenvalue)")

plt.show()

# SAVE LARGEST REAL
File("eigenmodes_tfm/eigenmodesu1_real.pvd") << u1real
File("eigenmodes_tfm/eigenmodesu2_real.pvd") << u2real

# SAVE LARGEST IMAGINARY
File("eigenmodes_tfm/eigenmodesu1_imag.pvd") << u1imag
File("eigenmodes_tfm/eigenmodesu2_imag.pvd") << u2imag

NameError: name 'A_sparray' is not defined

In [ ]:
# 1/
# SIMPLE two-fluid model: Holmas, Bonzanini (main)

# The interfacial and wall shear stresses in (A.2) are modelled using standard single-phase pipe friction relations (Haaland, 1983).
# https://fenicsproject.org/pub/tutorial/html/._ftut1017.html

# Libraries
from fenics import *
from dolfin import *
from mshr import *
from IPython.display import clear_output
from scipy.optimize import brenth
from scipy.optimize import fsolve
from scipy.linalg import eig
from scipy.sparse.linalg import eigs
from scipy.interpolate import interp1d
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from matplotlib import rc
from math import pi
from scipy.sparse import csr_matrix
from numpy import linalg as LA
from scipy import sparse

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
import time
import math
import ufl      
%matplotlib inline

# Phase properties
c2               = 293.43                 # Sanderse 2017
u4eq             = 1e5                    # Pa, outlet pressure (equilibrium state)
rho1             = 1000                   # kg m^-3, liquid density (Montini)
rho2             = u4eq/pow (c2, 2)       # kg m^-3, gas density (Sanderse)
mu1              = 8.9e-4                 # Pa s, liquid viscosity
mu2              = 1.8e-5                 # Pa s, gas viscosity
sigma            = 0.072                  # Montini 2011
rho              = (rho1 - rho2)/rho1     # Bonzanini 2017

# Gravity and driving force
g                = 9.8 # m/s^2
force            = 74.23 #Hendrix

# Configuration
d                = 0.078                  # 0.022              # m, diameter
L                = 1                      # m, pipe length
betadeg          = 0                      # (Taitel 1976)
beta             = pi*betadeg/180
Area             = pi*pow (abs(d), 2.)/4.      # mˆ2, total area
S                = pi*d
roughness        = 1e-8                   # m, pipe roughness

# superficial velocities (check well-posedness [as a function])
j1         = 0.5                #0.80 # liquid superficial velocity
j2         = 6.908              # 1.53 # gas superficial velocity
us         = j1 + j2            # velocity of the mixture
Re1s       = rho1*j1*d/mu1

# artifical diffusion coefficients (depend on superficial velocities and stability analysis)
E11        = 1e-3               #0.001 
E22        = 1e-2               #0.01 -------maior, estabiliza

# FUNCTIONS
# stratification angle
def gamma (u1):
    return 2*(pi*(1. - u1) + pow (abs(3.*pi/2.), (1./3.))*(2.*u1 - 1. + pow (abs (1. - u1), (1./3.))- pow (abs(u1), (1/3))) - (1./200.)*u1*(1. - u1)*(2.*u1 - 1.)*(1. + 4.*(pow (abs (u1), 2.) + pow (abs(1.- u1), 2.))))
# liquid holdup
def alpha1 (u1):
    return 1 - u1

# liquid phase velocity
def u3 (u1):
    return j1/(1. - u1)
# liquid phase velocity
def u3b (j1):
    return j1/(1. - u1)
# gas phase velocity
def u2 (u1):
    return j2/u1

# liquid sectional area
def A1 (gamma):
    return (d**2)/4*(gamma/2 - np.sin (gamma/2)*np.cos (gamma/2))
# gas sectional area
def A2 (gamma):
    return (d**2)/4*(pi - gamma/2 + np.sin (gamma/2)*np.cos (gamma/2))
# liquid wetted perimeter
def S1 (gamma):
    return d*gamma/2.
# gas wetted perimeter
def S2 (gamma):
    return d/2.*(2.*pi - gamma)
# interface wetted perimeter
def Si (gamma):
    return d*np.sin (gamma/2)

# liquid hydraulic diameter 
def dh1 (A1, S1):
    return 4.*A1/S1
# gas hydraulic diameter 
def dh2 (A2, S2, Si):
    return 4.*A2/(S2 + Si)
# critic diameter
def Dc (gamma):
    return g*np.cos (beta)*pi*d/(4*np.sin (gamma/2))

# liquid Reynolds number
def Re1 (u3, dh1):
    return rho1*u3*dh1/mu1
# gas Reynolds number
def Re2 (u2, dh2):
    return u4eq/pow ((c2), 2)*u2*dh2/mu2

# liquid shear stress
def tauw1 (f1, u3):
    return f1*rho1*u3*abs (u3)/2.
# gas shear stress
def tauw2 (f2, u2):
    return f2*u4eq/pow (c2, 2)*u2*abs (u2)/2.
# interface shear stress
def tau21 (fi, u3, u2):
    return fi*u4eq/pow (c2, 2)*(u2 - u3)*abs (u2 - u3)/2.
# interface shear stress
def tau12 (fi, u3, u2):
    return fi*u4eq/pow (c2, 2)*(u3 - u2)*abs (u3 - u2)/2.

# liquid friction factor 1
def f1 (Re1, dh1):
    Af1 = pow (2.457*np.log(pow(pow(abs(7/Re1), 0.9) + 0.27*roughness/dh1, - 1)), 16)
    Bf1 = pow (abs (37530/Re1), 16)
    return 2*pow (pow (abs (8/Re1), 12)+pow (abs (Af1 + Bf1), - 1.5), 1/12)

# gas friction factor 2
def f2 (Re2, dh2):
    Af2 = pow (2.457*np.log(pow(pow(abs(7/Re2), 0.9) + 0.27*roughness/dh2, - 1)), 16)
    Bf2 = pow (abs (37530/Re2), 16)
    return 2*pow (pow (abs (8/Re2), 12)+pow (abs (Af2 + Bf2), - 1.5), 1/12)

# interfacial friction
def fi (f2):
    return max (f2, 0.014)

# steady state momentum equation
def equilibrium1 (u1): 
    func = - (rho1 - u4eq/pow (c2, 2))*g*np.sin (beta) - tauw1 (f1 (Re1 (u3 (u1), dh1 (A1 (gamma (u1)), S1 (gamma (u1)))), dh1 (A1 (gamma (u1)), S1 (gamma (u1)))), u3 (u1))*S1 (gamma (u1))/A1 (gamma(u1)) + tauw2 (f2 (Re2 (u2 (u1), dh2 (A2 (gamma (u1)), S2 (gamma (u1)), Si (gamma (u1)))), dh2 (A2 (gamma (u1)), S2 (gamma (u1)), Si (gamma (u1)))), u2 (u1))*S2 (gamma (u1))/A2(gamma (u1)) + tau21 (fi (f2 (Re2 (u2 (u1), dh2 (A2 (gamma (u1)), S2 (gamma (u1)), Si(gamma (u1)))), dh2 (A2 (gamma (u1)), S2 (gamma (u1)), Si (gamma (u1))))), u3 (u1), u2 (u1))*Si(gamma (u1))*((1/A1 (gamma (u1))) + (1/A2 (gamma (u1))))
    return func
# print ("Local equilibrium theory")

lima    = 1e-15
limb    = 1 - lima
u1eq    = brenth (equilibrium1, lima, limb)
print("alpha2eq = ", u1eq)

# parameters used in boundary conditions
u1_0     = u1eq
u2_0     = u2 (u1eq)
u3_0     = u3 (u1eq)
u4_0     = u4eq

# messages for boundary conditions just at the entrance, but not the actual base flow)
print ("Uniform initial conditions phi (t, x) for both incompressible phases")
print ("Gas void fraction    : u1_0 (0, x) = ", u1_0, "[-]")
print ("Gas velocity         : u2_0 (0, x) = ", u2_0, "[m/s]")
print ("Liquid velocity      : u3_0 (0, x) = ", u3_0, "[m/s]")
print ("Interfacial pressure : u4_0 (0, x) = ", u4_0, "Pa")

print (" ")

print ("Boundary conditions phi (t, x) for both incompressible phases")
print ("Gas void fraction    : u1_0 (t, 0) = ", u1_0, "[-]")
print ("Gas velocity         : u2_0 (t, 0) = ", u2_0, "[m/s]")
print ("Liquid velocity      : u3_0 (t, 0) = ", u3_0, "[m/s]")
print ("Interfacial pressure : u4_0 (t, L) = ", u4_0, "Pa")

print (" ")

print ("Liquid holdup    = ", 1 - u1_0, "[-]")
print ("Liquid velocity  = ", u3_0, "[m/s]")


In [6]:
# 2

# 2/6
# Choose linear algebra backend    
parameters["linear_algebra_backend"] = "PETSc"

# Allow approximating values for points that may be generated outside
# of domain (because of numerical inaccuracies)
parameters["allow_extrapolation"] = True
parameters["refinement_algorithm"] = "plaza_with_parent_facets"

# Conditional
# ufl.algorithms.apply_derivatives.CONDITIONAL_WORKAROUND = True  

# Form compiler options
parameters ["form_compiler"]["optimize"]          = True
parameters ["form_compiler"]["cpp_optimize"]      = True
parameters ["form_compiler"]["quadrature_degree"] = 2

ffc_options = {"optimize":                          True, \
               "eliminate_zeros":                   True, \
               "precompute_basis_const":            True, \
               "precompute_ip_const":               True}

# Define constants
var1bc1   = 1 - u1_0                    # alpha1
var2bc1   = u3_0                        # u1

# Define space discretization properties
xmin      = 0
xmax      = L
nx        = 40                           # no. of elements 100 to 1600
p         = 1                              # degree of FEM
mesh      = IntervalMesh (nx, xmin, xmax)
deltax    = d/2.                           # D/2 Bonzanini #0.256*D holmas

# Save mesh
File ("sanderse/mesh.xml") << mesh

# Define funcion spaces
V1         = FiniteElement ('Lagrange', mesh.ufl_cell(), degree = p)
V2         = FiniteElement ('Lagrange', mesh.ufl_cell(), degree = p)
element    = MixedElement ([V1, V2])
V          = FunctionSpace (mesh, element)

# Define test and trial functions
v1, v2     = TestFunctions (V)

var        = Function (V, name = "Variables at current step")
var1, var2 = split (var)

dvar       = TrialFunction (V)

# Define boundary condition
var1_bc = Expression ("var1bc1", degree = p, var1bc1 = var1bc1)
var2_bc = Expression ("var2bc1", degree = p, var2bc1 = var2bc1)

# Sub domain for Dirichlet boundary condition
def right (x, on_boundary): 
    return x[0] > (1.0 - DOLFIN_EPS)
def left (x, on_boundary): 
    return x[0] < DOLFIN_EPS

bc1     = DirichletBC(V.sub (0), var1_bc, left, "pointwise")
bc3     = DirichletBC(V.sub (1), var2_bc, left, "pointwise")
bcs     = [bc1, bc3]


# # Define expressions used in weak form
# # k       = Expression ("dt", degree = p, dt = dt)
# E11     = Expression ("E11", degree = p, E11 = E11)
# E22     = Expression ("E22", degree = p, E22 = E22)


NameError: name 'parameters' is not defined

In [7]:
# 3

#Define constants and expressions
Area  = Constant (Area)
us  = Constant (us)

d         = Constant (d)
roughness = Constant (roughness)
rho1  = Constant (rho1)
rho2  = Constant (rho2)
rho   = Constant (rho)
g     = Constant (g)
# k     = Constant (dt)
# force = Constant (force)
beta1 = Expression ("beta*x[0]", degree = p, beta = beta)              

# For UFL
def Max (a, b): return (a + b + abs (a - b))/Constant (2)
def Min (a, b): return (a + b - abs (a - b))/Constant (2)

# Define functions used in weak form
def A11 (var1, var2):
    term1 = var2
    return term1

def A12 (var1, var2):
    term2 = var1
    return term2

def A21 (var1, var2):
    gamma = (2*(pi*( (var1)) + pow (((3*pi + DOLFIN_EPS)/2), 1/3)*(1 - 2*( (var1)) + pow ((var1) + DOLFIN_EPS, 1/3) - pow ((1-var1) + DOLFIN_EPS, 1/3))))
    S21   = d*sin (gamma/2)
    term3  = 1/(1 - rho*var1)*(Area*cos (beta1)*(1 - var1)*g*rho/S21 + (rho - 1)* pow (abs(var2)+DOLFIN_EPS, 2) - ((rho2/rho1)*(us - var1*var2)*(us - (2 - var1)*var2))/pow (abs(1 - var1)+ DOLFIN_EPS, 2))
    return term3
    
def A22 (var1, var2):
    term4  = 1/(1 - rho*var1)*((var2*(1 - 2*var1+rho*var1)) - ((2*rho2*var1*(us - var1*var2)/rho1)/(1 - var1)))
    return term4

def D2 (var1, var2):
    gamma = (2*(pi*( (var1)) + pow (((3*pi + DOLFIN_EPS)/2), 1/3)*(1 - 2*( (var1)) + pow ((var1) + DOLFIN_EPS, 1/3) - pow ((1-var1) + DOLFIN_EPS, 1/3))))    
    S1    = d*gamma/2
    S2    = d/2*(2*pi - gamma)
    S21   = d*sin (gamma/2)
    
    dh1   = 4*var1*Area/S1
    dh2   = 4*(1-var1)*Area/(S2 + S21)
    
    vel2  = (us - var1*var2)/(1 - var1)
    
    Re1   = rho1*var2*dh1/mu1
    Re2   = rho2*vel2*dh2/mu2
    
    f1    = 0.046*pow(abs(var2*dh1*rho1/mu1)+DOLFIN_EPS,-0.2)
    f2    = 0.046*pow(abs(vel2*dh2*rho2)+DOLFIN_EPS, - 0.2)

    f21    = Max (f2, 0.014)
        
    tau1  = f1*rho1*var2*abs (var2)/2.
    tau2  = f2*rho2*vel2*abs (vel2)/2.
    tau21 = f21*rho2*(vel2 - var2)*abs (vel2 - var2)/2.
    
    term5 = 1/(1 - rho*var1)*((var1*(tau2*S2 - tau1*S1) + tau21*S21)/(rho1*var1*Area) + (1 - var1)*g*rho*sin(beta1))
    return term5


NameError: name 'Constant' is not defined

In [8]:
# 4/

F = -(A11 (var1, var2)*v1*Dx(var1,0))*dx - 1*A12 (var1, var2)*(v1*Dx(var2,0))*dx + 1*(inner(E11*grad(var1), grad(v1)))*dx - \
1*A21 (var1, var2)*v2*Dx(var1,0)*dx - 1*(A22 (var1, var2)*v2*Dx(var2,0))*dx + 1*inner(E22*grad(var2), grad(v2))*dx + 1*D2 (var1, var2)*v2*dx


# # Define weak form (semi distretised with artificial difussion)
# F = -(A11 (var1, var2)*v1*Dx(var1,0))*dx - 1*A12 (var1, var2)*(v1*Dx(var2,0))*dx + \
# 1*(inner(E11*grad(var1), grad(v1)))*dx - 1*A21 (var1, var2)*v2*Dx(var1,0)*dx - \
# 1*(A22 (var1, var2)*v2*Dx(var2,0))*dx + 1*inner(E22*grad(var2), grad(v2))*dx + 1*D2 (var1, var2)*v2*dx

# Auxiliary
L = Constant(0)*v1*dx + Constant(0)*v2*dx

# Computation of the Jacobian using Gateaux derivativ
dF = derivative (F, var, dvar)

# Assemble stiffness form    
A = PETScMatrix()
b = PETScVector()

assemble_system(dF, L, bcs, A_tensor = A, b_tensor = b)

A_mat = as_backend_type(A).mat()

# Transform to numpy array
A_sparray = csr_matrix(A_mat.getValuesCSR()[::-1], shape = A_mat.size)

# Print array
print("A_sparray             =", A_sparray)

# Plot array
plt.figure (3, figsize = (12, 8))
plt.spy(A_sparray)
plt.grid (True, which = "both")
# plt.rcParams ['figure.figsize'] = [12, 8]

plt.show

# Eliminate nan
A_sparray = A_sparray.todense()
# A_sparray = np.nan_to_num (A_sparray)
A_sparray = sparse.csr_matrix(A_sparray)

# Print array
print("A_sparray without nan =", A_sparray)

# Eigenvalue problem

Acomplex_sparse = A_sparray.dot (1j)

Aeval           = A_sparray.toarray ()
# Aeval           = np.nan_to_num (Aeval)

Acomplex        = Aeval.dot (1j)

condnumber = LA.cond(Aeval)
print("Condition number :", condnumber)

#The array v of eigenvectors may not be of maximum rank, 
#that is, some of the columns may be linearly dependent, 
#although round-off error may obscure that fact. 
#If the eigenvalues are all different, then theoretically the eigenvectors are linearly independent.
# TUTORIALS
# https://fenicsproject.org/qa/13796/formulating-an-eigenvalue-problem/
# https://fenicsproject.org/qa/8680/solving-generalized-eigenvalues-finite-element-stiff-matrix/

vals, vecs = eig ( - Acomplex, overwrite_a = True)

# vals, vecs = eigs ( - Acomplex_sparse) #tol = 1e-10

# Eigenvalues
imagvals = vals.imag
realvals = vals.real

print("realvals = ", vals.real)
print("imagvals = ", vals.imag)

# Largest eigenvalues and eigenvectors localization
localmaxreal = np.where(vals.real == vals.real.max())
localmaximag = np.where(vals.imag == vals.imag.max())

print("localmaxreal =", localmaxreal)
print("localmaximag =", localmaximag)

eigenvector_real = vecs.real[localmaxreal[0]]
eigenvector_imag = vecs.imag[localmaximag[0]]

print("eigenvector_real =", eigenvector_real[0])
print("eigenvector_imag =", eigenvector_imag[0])

# Characteristics for plots
liststyles       = ["--", "-", "-.", "."]
listcolor        = ["k", "g", "b", "r", "none"]
listmarkers      = ["s", "o", "^", ">", "<", "p"]

# Plot eigenspectra 1
fig, ax = plt.subplots ()
area = 50

ax.scatter (realvals, imagvals, s = area, marker = listmarkers [0], color = listcolor [4], edgecolors = listcolor [0], linewidths = 1.5, alpha = 0.5)
ax.set_xscale ('symlog', linthreshx = 1e-4)

plt.rcParams ['figure.figsize'] = [12, 8]
# leg1 = ax.legend (loc = 'upper right', frameon = True, fontsize = 14);
plt.grid (True, which = "both")
plt.xlabel ('Re $[\omega]$ [1/s]', fontsize = 18)
plt.ylabel ('Im $[\omega]$ [1/s]', fontsize = 16)
        
matplotlib.rc ('xtick', labelsize = 18)     
matplotlib.rc ('ytick', labelsize = 18)

# plt.ylim (( - 0.002 , 0.002))
# plt.xlim (( 1e-3, 1e1))
plt.show ()    

# Plot eigenspectra 2
fig, ax = plt.subplots ()
area = 50

ax.scatter (realvals, imagvals, s = area, marker = listmarkers [0], color = listcolor [4], edgecolors = listcolor [0], linewidths = 1.5, alpha = 0.5)
# ax.set_xscale ('symlog',  linthreshx = 1e-18)

ax.xaxis.set_minor_locator(MultipleLocator(1e-15))
plt.rcParams ['figure.figsize'] = [12, 8]
# leg1 = ax.legend (loc = 'upper right', frameon = True, fontsize = 14);
plt.grid (True, which = "both")
plt.xlabel ('Re $[\omega]$ [1/s]', fontsize = 18)
plt.ylabel ('Im $[\omega]$ [1/s]', fontsize = 16)
        
matplotlib.rc ('xtick', labelsize = 18)     
matplotlib.rc ('ytick', labelsize = 18)

# plt.ylim (( - 0.002 , 0.002))
plt.xlim (( - 5*realvals[localmaxreal[0][0]], 5*realvals[localmaxreal[0][0]])) #primeriro teste
# plt.xlim (( - 1e-10, 1e-10))

plt.show ()    

# Eigenvectors
# Plot real eigenvectors
ureal  = Function(V)
ureal.vector()[:] = eigenvector_real[0] #len(vecs)-1 
u1real, u2real = ureal.split()

# Plot eigenfunction
plt.figure (3, figsize = (12, 8))
plot(u1real, wireframe = True, title = 'Eigenfunction')
plt.grid (True, which = "both")
plt.xlabel("L")
plt.ylabel("rx (Eigenvector of the largest real eigenvalue)")

# Plot eigenfunction
plt.figure (3, figsize = (12, 8))
plot(u2real, wireframe = True, title = 'Eigenfunction')
plt.grid (True, which = "both")
plt.xlabel("L")
plt.ylabel("rx (Eigenvector of the largest real eigenvalue)")

# Plot imaginary eigenvectors
uimag = Function(V)
uimag.vector()[:] = eigenvector_imag[0] #eigenfunction)
u1imag, u2imag = uimag.split()

# Plot eigenfunction
plt.figure (4, figsize = (12, 8))
plot(u1imag, wireframe = True, title = 'Eigenfunction')
plt.grid (True, which = "both")
plt.xlabel("L")
plt.ylabel("cx (Eigenvector of the largest imaginary eigenvalue)")

plt.figure (4, figsize = (12, 8))
plot(u2imag, wireframe = True, title = 'Eigenfunction')
plt.grid (True, which = "both")
plt.xlabel("L")
plt.ylabel("cx (Eigenvector of the largest imaginary eigenvalue)")

plt.show()

# SAVE LARGEST REAL
File("sanderse/eigenmodesu1_real.pvd") << u1real
File("sanderse/eigenmodesu2_real.pvd") << u2real

# SAVE LARGEST IMAGINARY
File("sanderse/eigenmodesu1_imag.pvd") << u1imag
File("sanderse/eigenmodesu2_imag.pvd") << u2imag

NameError: name 'A11' is not defined

In [9]:
# 5/

# from numpy.random import rand
# var.vector().set_local(rand(dvar.vector().size()))
# dvar.vector().apply("")
    
# solve stationary state
# Compute solution
problem = NonlinearVariationalProblem (F, var, bcs = bcs, J = dF, form_compiler_parameters = ffc_options)
solver  = NonlinearVariationalSolver (problem)
prm     = solver.parameters
    
info(prm, True)
    
prm ['nonlinear_solver'] = 'newton'
prm ['print_matrix']     = False #True
prm ['print_rhs']        = False #True
prm ['symmetric']        = False #True
    
prm ['newton_solver']['absolute_tolerance']      = 1e-20 #1E-8
prm ['newton_solver']['convergence_criterion']   = 'residual' #'residual' 'incremental'
# prm ['newton_solver']['error_on_nonconvergence'] = True
prm ['newton_solver']['linear_solver']           = 'umfpack' # 'bicgstab' 'cg' 'gmres' 'minres' 'petsc' 'richardson' 'superlu_dist' 'tfqmr' 'umfpack'
prm ['newton_solver']['maximum_iterations']      = 1000
#     prm ['newton_solver']['preconditioner']          = 'ilu' # 'ilu' 'icc' 'petsc_amg' 'sor'
prm ['newton_solver']['relative_tolerance']      = 1e-18
prm ['newton_solver']['relaxation_parameter']    = 1.0
prm ['newton_solver']['report']                  = True

# prm ['newton_solver']['krylov_solver']['absolute_tolerance']       = 1e-3 #1E-9
# #     prm ['newton_solver']['krylov_solver']['error_on_nonconvergence']  = True
# prm ['newton_solver']['krylov_solver']['maximum_iterations']       = 10000 # 500000
# prm ['newton_solver']['krylov_solver']["monitor_convergence"]      = True
# prm ['newton_solver']['krylov_solver']["nonzero_initial_guess"]    = False #False
# prm ['newton_solver']['krylov_solver']['relative_tolerance']       = 1e-3
# prm ['newton_solver']['krylov_solver']['report']                   = True
    
prm ['newton_solver']['lu_solver']['report']    = True
prm ['newton_solver']['lu_solver']['symmetric'] = False
prm ['newton_solver']['lu_solver']['verbose']   = True

PROGRESS = 1
set_log_level (PROGRESS)

solver.solve ()
    
(var1, var2) = var.split (deepcopy = True)
    
# Plot solution
plt.figure (5, figsize = (8, 4))
    #plt.clf()
plot (var1, wireframe = True, title = "Liquid holdup")
    
plt.figure (6, figsize = (8, 4))
    #plt.clf()
plot (var2, wireframe = True, title = "Liquid velocity")
    
# Update previous solution
# var_n.assign (var)
    
# Save solution
ff1 << var1
ff2 << var2

# Show time of the program's execution
start_time  = time.time ()
if __name__ == '__main__':
    print("Time of the program's execution: %s seconds " % (time.time () - start_time))
    
# solvers and preconditioners
# https://github.com/hplgit/fenics-tutorial/issues/33


NameError: name 'NonlinearVariationalProblem' is not defined

In [10]:
# Define time discretization properties
T         = 1.0                         # final time Holmas (1, 4, 7 cycles)
CFL       = 0.978                       # 0.7 Holmas, 0.978 Van Zwieten
dt        = CFL*deltax/(var2bc1)        # Van Zwieten & Smith /lambda_max Holmas (maximum modulus of the eigenvalues of the Jacobian of (4.1) at time step n)
num_steps = int ( round (T/dt))
                
print('T         =', T)
print('dt        =', dt)
print('num_steps =', num_steps)


# For initial conditions
k_wave     = 2*pi  #Hendrix
omega_wave = 8.484 #Hendrix
t          = 0

# Define initial condition
class InitialConditions (UserExpression):
#     def __init__ (self, **kwargs):
#         super (InitialConditions, self).__init__(**kwargs)
    def eval (self, values, x):
        values[0] = var1bc1 + 0.01*sin(k_wave*x[0] - omega_wave*t) #u1bc1
        values[1] = var2bc1 + 7e-3*sin(k_wave*x[0] - omega_wave*t) # sin(x-pi/3)
    def value_shape (self):
        return (2,)

var_ic         = InitialConditions (degree = p)
var_n          = interpolate (var_ic, V) 
var1_n, var2_n = split (var_n)

# Plot initial conditions
plt.figure (1, figsize = (8, 4))
plt.grid (True, which = "both")
plt.xlim (xmin, xmax)
plot (var1_n, wireframe = True, title = "Initial liquid holdup")

plt.figure (2, figsize = (8, 4))
plt.grid (True, which = "both")
plt.xlim (xmin, xmax)
plot (var2_n, wireframe = True, title = "Initial liquid velocity")

plt.show

NameError: name 'deltax' is not defined

In [ ]:
# 6/

# transient
# Define weak form (discretised system)
F = ((var1 - var2_n) / k)*v1*dx + (A11 (var1, var2)*v1*Dx(var1,0))*dx + A12 (var1, var2)*(v1*Dx(var2,0))*dx + (inner(E11*grad(var1), grad(v1)))*dx + force*v1*dx\
+ ((var2 - var2_n) / k)*v2*dx + A21 (var1, var2)*v2*Dx(var1,0)*dx + (A22 (var1, var2)*v2*Dx(var2,0))*dx + inner(E22*grad(var2), grad(v2))*dx - D2 (var1, var2)*v2*dx + force*v2*dx

# Define Jacobian
dF = derivative (F, var, dvar)

# Define files
ff1 = File ("sanderse/var1.pvd", "compressed")
ff2 = File ("sanderse/var2.pvd", "compressed")

# Iterative process
t = 0
for n in range (num_steps):

    # Update current time
    t += dt
    #u_D.t = t
    
    # Print progress
    clear_output ()
    print("Iteration :", t/dt, "of", num_steps)
    print("Time      :", t, "s")
    
    # Compute solution
    problem = NonlinearVariationalProblem (F, var, bcs = bcs, J = dF)
    solver  = NonlinearVariationalSolver (problem)
    prm     = solver.parameters
    
    info(prm, True)
    
    prm ['nonlinear_solver'] = 'newton'
    prm ['print_matrix']     = False #True
    prm ['print_rhs']        = False #True
    prm ['symmetric']        = False #True
    
    prm ['newton_solver']['absolute_tolerance']      = 1e-20 #1E-8
    prm ['newton_solver']['convergence_criterion']   = 'residual' #'residual' 'incremental'
    prm ['newton_solver']['error_on_nonconvergence'] = True
    prm ['newton_solver']['linear_solver']           = 'umfpack' # 'bicgstab' 'cg' 'gmres' 'minres' 'petsc' 'richardson' 'superlu_dist' 'tfqmr' 'umfpack'
    prm ['newton_solver']['maximum_iterations']      = 1000
#     prm ['newton_solver']['preconditioner']          = 'ilu' # 'ilu' 'icc' 'petsc_amg' 'sor'
    prm ['newton_solver']['relative_tolerance']      = 1e-18
    prm ['newton_solver']['relaxation_parameter']    = 1.0
    prm ['newton_solver']['report']                  = True

    prm ['newton_solver']['krylov_solver']['absolute_tolerance']       = 1e-3 #1E-9
#     prm ['newton_solver']['krylov_solver']['error_on_nonconvergence']  = True
    prm ['newton_solver']['krylov_solver']['maximum_iterations']       = 10000 # 500000
    prm ['newton_solver']['krylov_solver']["monitor_convergence"]      = True
    prm ['newton_solver']['krylov_solver']["nonzero_initial_guess"]    = False #False
    prm ['newton_solver']['krylov_solver']['relative_tolerance']       = 1e-3
    prm ['newton_solver']['krylov_solver']['report']                   = True
    
    prm ['newton_solver']['lu_solver']['report']    = True
    prm ['newton_solver']['lu_solver']['symmetric'] = False
    prm ['newton_solver']['lu_solver']['verbose']   = True

    PROGRESS = 1
    set_log_level (PROGRESS)
    
    solver.solve ()
    
    (var1, var2) = var.split (deepcopy = True)
    
    # Plot solution
    plt.figure (1, figsize = (8, 4))
    #plt.clf()
    plot (var1, wireframe = True, title = "Liquid holdup")
    
    plt.figure (2, figsize = (8, 4))
    #plt.clf()
    plot (var2, wireframe = True, title = "Liquid velocity")
    
    # Update previous solution
    var_n.assign (var)
    
    # Save solution
    ff1 << var1
    ff2 << var2

# Hold plot
plt.show ()

# Show time of the program's execution
start_time  = time.time ()
if __name__ == '__main__':
    print("Time of the program's execution: %s seconds " % (time.time () - start_time))
    